In [1]:
# prepare data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json
import glob
import operator
from IPython.display import display, HTML

In [2]:
%load_ext autoreload
%autoreload 1

from shared_globals_functions import *
%aimport shared_globals_functions

In [3]:
t_name='t1'
merge_list=['t1.1', 't1.2', 't1.3', 't1.4']

cust_dir = 'test'

tdidf_max_df=1.0 # defaults to 1.0 (float), can be an int too
tdidf_min_df=4
use_tfidfvectorizer = True
use_stopwords = True
use_extended_stopwords = False

umap_metric='hellinger'
umap_ncomponents=2
umap_random_state=42
umap_min_dist=0 #0.1
umap_n_neighbors=30 #10, 30

min_cluster=10
max_cluster=30
clust_int=3

ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50

#tf-idf vectorizer
#max_df=0.3
#min_df=

#kmeans
#n_clusters
#n_init
#max_iter

In [4]:
# Parameters
cust_dir = "ar3/"
use_tfidfvectorizer = True
tdidf_max_df = 1.0
tdidf_min_df = 4
use_stopwords = True
use_extended_stopwords = False
umap_metric = "hellinger"
umap_ncomponents = 2
umap_random_state = 42
umap_min_dist = 0
umap_n_neighbors = 30
min_cluster = 10
max_cluster = 50
clust_int = 3
ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50
t_name = "t2.3.6"
merge_list = ["t2.3.3", "t2.3.5"]


In [5]:
testset = from_disk(t_name)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.3.6.pkl


In [6]:
len(testset[testset['topics'].str.len() > 0])

28395

In [7]:
if len(merge_list)>0:
    t_name = f'{t_name}+{"+".join([re.sub(r"t", "", tid) for tid in merge_list])}'
    print(t_name)

t2.3.6+2.3.3+2.3.5


In [8]:
testset_merge = create_merge(testset, merge_list, merge_to_one=True)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.3.3.pkl


reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.3.5.pkl


In [9]:
len(testset_merge[testset_merge['topics'].str.len() > 0])

28395

In [10]:
df = testset_merge.merge(testset, on=['name'], how='left', indicator=True)

In [11]:
df[df['_merge']!='both']

,name,topics_x,topics_y,_merge


In [12]:
df[df['topics_x'].str.len() == 0]

,name,topics_x,topics_y,_merge


In [13]:
if len(merge_list)>0:
    testset = testset_merge

In [14]:
import pandas as pd
import umap
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [15]:
orig_testset = testset

In [16]:
my_stop_words=None
if use_stopwords:
    my_stop_words='english'
    if use_extended_stopwords:
        print("todo")

if use_tfidfvectorizer:
    vectorizer = TfidfVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)
else:
    vectorizer = CountVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)

print(f"total repos {len(testset[testset['topics'].str.len() > 0])}")
testset = testset[testset['topics'].str.len() > 0].reset_index()

if isinstance(testset['topics'][0], list):
    testset['topics'] = testset['topics'].apply(lambda x: ' '.join(x))
    print("topics came as list ... consider joining it before")

cr_start()
word_doc_matrix = vectorizer.fit_transform(testset['topics'])
runtime_vectorizer = cr_getrt()

total repos 28395
topics came as list ... consider joining it before


In [17]:
runtime_vectorizer

0.22918415069580078

In [18]:
word_doc_matrix

<28395x4247 sparse matrix of type '<class 'numpy.float64'>'
	with 385877 stored elements in Compressed Sparse Row format>

In [19]:
class ProgressWriter:
    def write(self, text):
        print("custom progress", n, total)

    def flush(self):
        pass

#tqdm_kwds = {"file": "progress_writer", "disable": False }
tqdm_kwds = {"disable": False }   


In [20]:
%%time
# try also with default parameters for n_neighbors and min_dist

cr_start()

embedding = umap.UMAP(
    n_neighbors=umap_n_neighbors,
    min_dist=umap_min_dist,
    n_components=umap_ncomponents, metric=umap_metric,
    tqdm_kwds=tqdm_kwds,
    random_state=umap_random_state,
).fit(word_doc_matrix)

runtime_umap = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/umap/umap_.py:125: UserWarning: A few of your vertices were disconnected from the manifold.  This shouldn't cause problems.
Disconnection_distance = 1 has removed 307 edges.
It has only fully disconnected 5 vertices.
Use umap.utils.disconnected_vertices() to identify them.
  warn(


Epochs completed:   0%|            0/200 [00:00]

CPU times: user 4min 44s, sys: 8.51 s, total: 4min 53s
Wall time: 1min 37s


In [21]:
print(runtime_umap)

97.7764105796814


In [22]:
%%time

cr_start()
clusterable_embedding = embedding.transform(word_doc_matrix)
runtime_umap_transform = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Epochs completed:   0%|            0/30 [00:00]

CPU times: user 1min 7s, sys: 64.8 ms, total: 1min 7s
Wall time: 58.7 s


In [23]:
print(runtime_umap_transform)

58.670198917388916


In [24]:
umap_embedding = pd.DataFrame(embedding.embedding_, columns=['x', 'y']) # (umap_t)
umap_clusterable_embedding = pd.DataFrame(clusterable_embedding, columns=['x', 'y']) # (umap_t)

umap_embedding['name'] = testset['name']
umap_clusterable_embedding['name'] = testset['name']

to_disk_results(umap_embedding, f'umap_embedding_{t_name}', subdir=f"{cust_dir}")
to_disk_results(umap_clusterable_embedding, f'umap_clusterable_embedding_{t_name}', subdir=f"{cust_dir}")

writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//umap_embedding_t2.3.6+2.3.3+2.3.5.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//umap_clusterable_embedding_t2.3.6+2.3.3+2.3.5.pkl


https://hdbscan.readthedocs.io/en/latest/parameter_selection.html

Since we have seen that min_samples clearly has a dramatic effect on clustering, the question becomes: how do we select this parameter? The simplest intuition for what min_samples does is provide a measure of how conservative you want you clustering to be. The larger the value of min_samples you provide, the more conservative the clustering – more points will be declared as noise, and clusters will be restricted to progressively more dense areas. We can see this in practice by leaving the min_cluster_size at 60, but reducing min_samples to 1.

In [25]:
import hdbscan

def clusterapply_hdbscan(hdbscan_min_samples, min_cluster_size, runner_name):
    cr_start()
    labels = hdbscan.HDBSCAN(
        min_samples=hdbscan_min_samples,
        min_cluster_size=min_cluster_size,
    ).fit_predict(clusterable_embedding)
    runtime_hdb = cr_getrt()
    
    stats = {'type': "hdb",
             'testset': t_name,
             'version': td_version,
            }
    
    detail_stats = {'hdb_minclust': min_cluster_size,
                    'hdb_minsamp': hdbscan_min_samples,
                    'runtime': runtime_hdb,
                    'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                    'umap': f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}",
                    #'gtr': grtr,
                    #'gtr_rel': grtr_rt
                   }
    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'tot_anom':no_anomalies, 'no_lab':missing_clabel, 'anom':identified_anom})
    prediction.update(detail_stats)
    res.append(prediction.copy())
    display(pd.DataFrame.from_dict(res))
    return labels

res = []
hdb_labels_results = pd.DataFrame()
hdb_labels_results['name'] = testset['name'].copy()


for cs in range(cs_min, cs_max, cs_int): #50, 500
    for ss in range(ss_min, ss_max, ss_int): #50 500
        runner_name = f'ss{ss}cs{cs}'
        hdb_labels_results[runner_name] = clusterapply_hdbscan(ss, cs, runner_name)

results = pd.DataFrame.from_dict(res)
res = []


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6+2.3.3+2.3.5,1,ss50cs50,50,50,1.073524,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6+2.3.3+2.3.5,1,ss50cs50,50,50,1.073524,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
1,hdb,t2.3.6+2.3.3+2.3.5,1,ss100cs50,50,100,0.502680,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6+2.3.3+2.3.5,1,ss50cs50,50,50,1.073524,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
1,hdb,t2.3.6+2.3.3+2.3.5,1,ss100cs50,50,100,0.502680,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
2,hdb,t2.3.6+2.3.3+2.3.5,1,ss150cs50,50,150,0.567822,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6+2.3.3+2.3.5,1,ss50cs50,50,50,1.073524,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
1,hdb,t2.3.6+2.3.3+2.3.5,1,ss100cs50,50,100,0.502680,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
2,hdb,t2.3.6+2.3.3+2.3.5,1,ss150cs50,50,150,0.567822,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
3,hdb,t2.3.6+2.3.3+2.3.5,1,ss200cs50,50,200,0.688992,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6+2.3.3+2.3.5,1,ss50cs50,50,50,1.073524,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
1,hdb,t2.3.6+2.3.3+2.3.5,1,ss100cs50,50,100,0.502680,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
2,hdb,t2.3.6+2.3.3+2.3.5,1,ss150cs50,50,150,0.567822,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
3,hdb,t2.3.6+2.3.3+2.3.5,1,ss200cs50,50,200,0.688992,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
4,hdb,t2.3.6+2.3.3+2.3.5,1,ss250cs50,50,250,0.763339,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."


...

,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6+2.3.3+2.3.5,1,ss50cs50,50,50,1.073524,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
1,hdb,t2.3.6+2.3.3+2.3.5,1,ss100cs50,50,100,0.502680,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
2,hdb,t2.3.6+2.3.3+2.3.5,1,ss150cs50,50,150,0.567822,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
3,hdb,t2.3.6+2.3.3+2.3.5,1,ss200cs50,50,200,0.688992,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
4,hdb,t2.3.6+2.3.3+2.3.5,1,ss250cs50,50,250,0.763339,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
...,...,...,...,...,...,...,...,...,...
72,hdb,t2.3.6+2.3.3+2.3.5,1,ss50cs450,450,50,0.452284,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
73,hdb,t2.3.6+2.3.3+2.3.5,1,ss100cs450,450,100,0.514827,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
74,hdb,t2.3.6+2.3.3+2.3.5,1,ss150cs450,450,150,0.586167,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
75,hdb,t2.3.6+2.3.3+2.3.5,1,ss200cs450,450,200,0.658317,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6+2.3.3+2.3.5,1,ss50cs50,50,50,1.073524,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
1,hdb,t2.3.6+2.3.3+2.3.5,1,ss100cs50,50,100,0.502680,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
2,hdb,t2.3.6+2.3.3+2.3.5,1,ss150cs50,50,150,0.567822,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
3,hdb,t2.3.6+2.3.3+2.3.5,1,ss200cs50,50,200,0.688992,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
4,hdb,t2.3.6+2.3.3+2.3.5,1,ss250cs50,50,250,0.763339,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
...,...,...,...,...,...,...,...,...,...
73,hdb,t2.3.6+2.3.3+2.3.5,1,ss100cs450,450,100,0.514827,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
74,hdb,t2.3.6+2.3.3+2.3.5,1,ss150cs450,450,150,0.586167,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
75,hdb,t2.3.6+2.3.3+2.3.5,1,ss200cs450,450,200,0.658317,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
76,hdb,t2.3.6+2.3.3+2.3.5,1,ss250cs450,450,250,0.690061,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6+2.3.3+2.3.5,1,ss50cs50,50,50,1.073524,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
1,hdb,t2.3.6+2.3.3+2.3.5,1,ss100cs50,50,100,0.502680,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
2,hdb,t2.3.6+2.3.3+2.3.5,1,ss150cs50,50,150,0.567822,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
3,hdb,t2.3.6+2.3.3+2.3.5,1,ss200cs50,50,200,0.688992,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
4,hdb,t2.3.6+2.3.3+2.3.5,1,ss250cs50,50,250,0.763339,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
...,...,...,...,...,...,...,...,...,...
74,hdb,t2.3.6+2.3.3+2.3.5,1,ss150cs450,450,150,0.586167,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
75,hdb,t2.3.6+2.3.3+2.3.5,1,ss200cs450,450,200,0.658317,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
76,hdb,t2.3.6+2.3.3+2.3.5,1,ss250cs450,450,250,0.690061,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
77,hdb,t2.3.6+2.3.3+2.3.5,1,ss300cs450,450,300,0.763637,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6+2.3.3+2.3.5,1,ss50cs50,50,50,1.073524,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
1,hdb,t2.3.6+2.3.3+2.3.5,1,ss100cs50,50,100,0.502680,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
2,hdb,t2.3.6+2.3.3+2.3.5,1,ss150cs50,50,150,0.567822,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
3,hdb,t2.3.6+2.3.3+2.3.5,1,ss200cs50,50,200,0.688992,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
4,hdb,t2.3.6+2.3.3+2.3.5,1,ss250cs50,50,250,0.763339,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
...,...,...,...,...,...,...,...,...,...
75,hdb,t2.3.6+2.3.3+2.3.5,1,ss200cs450,450,200,0.658317,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
76,hdb,t2.3.6+2.3.3+2.3.5,1,ss250cs450,450,250,0.690061,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
77,hdb,t2.3.6+2.3.3+2.3.5,1,ss300cs450,450,300,0.763637,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
78,hdb,t2.3.6+2.3.3+2.3.5,1,ss350cs450,450,350,0.870402,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6+2.3.3+2.3.5,1,ss50cs50,50,50,1.073524,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
1,hdb,t2.3.6+2.3.3+2.3.5,1,ss100cs50,50,100,0.502680,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
2,hdb,t2.3.6+2.3.3+2.3.5,1,ss150cs50,50,150,0.567822,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
3,hdb,t2.3.6+2.3.3+2.3.5,1,ss200cs50,50,200,0.688992,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
4,hdb,t2.3.6+2.3.3+2.3.5,1,ss250cs50,50,250,0.763339,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
...,...,...,...,...,...,...,...,...,...
76,hdb,t2.3.6+2.3.3+2.3.5,1,ss250cs450,450,250,0.690061,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
77,hdb,t2.3.6+2.3.3+2.3.5,1,ss300cs450,450,300,0.763637,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
78,hdb,t2.3.6+2.3.3+2.3.5,1,ss350cs450,450,350,0.870402,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."
79,hdb,t2.3.6+2.3.3+2.3.5,1,ss400cs450,450,400,0.910294,"rt_vect: 0.22918415069580078, min_df: 4, max_d...","hellinger, rt_umap_tf: 58.670198917388916, rt_..."


In [26]:
hdb_labels_results

,name,ss50cs50,ss100cs50,ss150cs50,ss200cs50,ss250cs50,ss300cs50,ss350cs50,ss400cs50,ss450cs50,...,ss450cs400,ss50cs450,ss100cs450,ss150cs450,ss200cs450,ss250cs450,ss300cs450,ss350cs450,ss400cs450,ss450cs450
0,google/chaps-linux,74,37,22,20,6,6,5,5,5,...,5,4,5,2,5,5,5,5,5,5
1,droidefense/engine,-1,15,12,11,6,6,5,5,5,...,5,4,5,2,5,5,5,5,5,5
2,wdgdc/Jarvis,-1,-1,7,5,4,4,3,3,3,...,3,2,2,2,3,3,3,3,3,3
3,PaloAltoNetworks/google-cloud-hub-spoke-tutorial,41,-1,-1,-1,6,6,5,5,5,...,5,4,-1,2,5,5,5,5,5,5
4,liwenhe1993/charts,-1,-1,35,26,6,6,5,5,5,...,5,4,5,2,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28390,sakhnyuk/javascript-interview-questions,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,2,2,-1,-1,-1,-1,-1
28391,mohammaduzair9/Face-Recognition,89,45,26,18,6,6,5,5,5,...,5,4,5,2,5,5,5,5,5,5
28392,deepinx/age-gender-estimation,-1,-1,-1,-1,6,6,5,5,5,...,5,4,5,2,5,5,5,5,5,5
28393,aws-cloudformation/aws-cloudformation-resource...,67,15,12,11,6,6,5,5,5,...,5,4,5,2,5,5,5,5,5,5


In [27]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/hdb_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'hdb_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'hdb_results', subdir=f"{cust_dir}")
to_disk_results(hdb_labels_results, f'hdb_labels_results_{t_name}', subdir=f"{cust_dir}")

saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_labels_results_t2.3.6+2.3.3+2.3.5.pkl


In [28]:

umap_embedding['topics'] = testset['topics']
umap_clusterable_embedding['topics'] = testset['topics']

In [29]:
import sklearn.cluster as cluster

def clusterapply_kmeans(testset, clustersize, runner_name, opt_comment=""):
    # Perform k-means clustering
    
    kmeans = cluster.KMeans(n_clusters=clustersize, random_state=42)
        
    if('y' in testset.columns):
        #print("assuming umap reduced testset here")
        testset = testset.copy()
        dropping_not_connected = len(testset[testset['y'].isnull()])
        testset = testset[testset['y'].notna()]
        cr_start()
        kmeans.fit(testset[['x', 'y']].to_numpy())
        runtime_kmeans = cr_getrt()
        opt_comment="-umapt"
    else:
        dropping_not_connected = 0
        cr_start()
        kmeans.fit(word_doc_matrix)
        runtime_kmeans = cr_getrt()

    
    stats = {'type': f"kmeans{opt_comment}",
             'testset': t_name,
             'version': td_version,
             'size': clustersize
            }
    
    detail_stats = {'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                'runtime': runtime_kmeans
               }
    
    if('y' in testset.columns):
        detail_stats['umap'] = f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}", #umap not important here
        stats['no_con'] = dropping_not_connected

    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'no_lab':missing_clabel})
    prediction.update(detail_stats)
    res.append(prediction.copy())
        
    display(pd.DataFrame.from_dict(res))
    return kmeans.labels_
    #testset.drop(['cluster_run'], axis=1)


In [30]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = umap_embedding[umap_embedding['y'].notna()]['name'].copy()

for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(umap_embedding, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix



,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,10,5,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.2714,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,10,5,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.271400,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
1,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,13,5,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.209875,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,10,5,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.271400,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
1,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,13,5,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.209875,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
2,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,16,5,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.247399,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,10,5,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.271400,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
1,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,13,5,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.209875,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
2,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,16,5,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.247399,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
3,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,19,5,19,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.376683,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,10,5,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.271400,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
1,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,13,5,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.209875,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
2,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,16,5,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.247399,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
3,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,19,5,19,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.376683,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
4,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,22,5,22,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.345729,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."


...

,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,10,5,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.271400,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
1,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,13,5,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.209875,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
2,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,16,5,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.247399,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
3,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,19,5,19,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.376683,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
4,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,22,5,22,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.345729,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
5,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,25,5,25,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.412730,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
6,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,28,5,28,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.448476,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
7,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,31,5,31,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.497677,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
8,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,34,5,34,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.520454,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
9,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,37,5,37,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.557126,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,10,5,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.271400,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
1,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,13,5,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.209875,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
2,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,16,5,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.247399,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
3,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,19,5,19,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.376683,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
4,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,22,5,22,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.345729,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
5,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,25,5,25,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.412730,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
6,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,28,5,28,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.448476,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
7,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,31,5,31,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.497677,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
8,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,34,5,34,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.520454,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
9,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,37,5,37,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.557126,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,10,5,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.271400,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
1,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,13,5,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.209875,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
2,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,16,5,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.247399,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
3,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,19,5,19,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.376683,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
4,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,22,5,22,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.345729,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
5,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,25,5,25,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.412730,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
6,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,28,5,28,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.448476,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
7,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,31,5,31,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.497677,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
8,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,34,5,34,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.520454,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
9,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,37,5,37,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.557126,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,10,5,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.271400,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
1,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,13,5,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.209875,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
2,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,16,5,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.247399,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
3,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,19,5,19,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.376683,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
4,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,22,5,22,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.345729,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
5,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,25,5,25,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.412730,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
6,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,28,5,28,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.448476,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
7,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,31,5,31,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.497677,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
8,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,34,5,34,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.520454,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
9,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,37,5,37,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.557126,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,10,5,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.271400,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
1,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,13,5,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.209875,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
2,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,16,5,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.247399,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
3,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,19,5,19,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.376683,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
4,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,22,5,22,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.345729,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
5,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,25,5,25,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.412730,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
6,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,28,5,28,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.448476,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
7,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,31,5,31,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.497677,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
8,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,34,5,34,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.520454,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."
9,kmeans-umapt,t2.3.6+2.3.3+2.3.5,1,37,5,37,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",0.557126,"(hellinger, rt_umap_tf: 58.670198917388916, rt..."


In [31]:
kmeans_labels_results

,name,10,13,16,19,22,25,28,31,34,37,40,43,46,49
0,google/chaps-linux,1,2,4,0,12,14,19,13,11,8,14,6,42,12
1,droidefense/engine,1,0,1,17,10,11,18,25,26,0,0,41,37,26
2,wdgdc/Jarvis,6,1,3,1,17,23,14,18,13,11,3,11,18,7
3,PaloAltoNetworks/google-cloud-hub-spoke-tutorial,1,12,4,0,12,5,11,14,24,25,8,21,22,22
4,liwenhe1993/charts,1,0,4,0,12,17,5,6,11,33,32,1,16,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28390,sakhnyuk/javascript-interview-questions,2,8,6,16,5,4,6,27,6,1,11,17,15,43
28391,mohammaduzair9/Face-Recognition,4,11,8,13,6,20,25,15,15,5,35,31,2,1
28392,deepinx/age-gender-estimation,0,2,5,8,6,20,25,15,15,5,1,12,38,38
28393,aws-cloudformation/aws-cloudformation-resource...,1,0,1,17,10,11,18,25,26,0,0,41,37,26


In [32]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/kmeans-umap_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans-umap_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans-umap_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans-umap_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_labels_results_t2.3.6+2.3.3+2.3.5.pkl


In [33]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = testset['name'].copy()


for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(testset, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix

,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6+2.3.3+2.3.5,1,10,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.437257


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6+2.3.3+2.3.5,1,10,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.437257
1,kmeans,t2.3.6+2.3.3+2.3.5,1,13,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.670915


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6+2.3.3+2.3.5,1,10,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.437257
1,kmeans,t2.3.6+2.3.3+2.3.5,1,13,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.670915
2,kmeans,t2.3.6+2.3.3+2.3.5,1,16,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.729734


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6+2.3.3+2.3.5,1,10,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.437257
1,kmeans,t2.3.6+2.3.3+2.3.5,1,13,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.670915
2,kmeans,t2.3.6+2.3.3+2.3.5,1,16,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.729734
3,kmeans,t2.3.6+2.3.3+2.3.5,1,19,19,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.995792


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6+2.3.3+2.3.5,1,10,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.437257
1,kmeans,t2.3.6+2.3.3+2.3.5,1,13,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.670915
2,kmeans,t2.3.6+2.3.3+2.3.5,1,16,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.729734
3,kmeans,t2.3.6+2.3.3+2.3.5,1,19,19,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.995792
4,kmeans,t2.3.6+2.3.3+2.3.5,1,22,22,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",2.332657


...

,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6+2.3.3+2.3.5,1,10,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.437257
1,kmeans,t2.3.6+2.3.3+2.3.5,1,13,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.670915
2,kmeans,t2.3.6+2.3.3+2.3.5,1,16,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.729734
3,kmeans,t2.3.6+2.3.3+2.3.5,1,19,19,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.995792
4,kmeans,t2.3.6+2.3.3+2.3.5,1,22,22,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",2.332657
5,kmeans,t2.3.6+2.3.3+2.3.5,1,25,25,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",2.756663
6,kmeans,t2.3.6+2.3.3+2.3.5,1,28,28,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.043608
7,kmeans,t2.3.6+2.3.3+2.3.5,1,31,31,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.811321
8,kmeans,t2.3.6+2.3.3+2.3.5,1,34,34,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.926773
9,kmeans,t2.3.6+2.3.3+2.3.5,1,37,37,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",4.188998


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6+2.3.3+2.3.5,1,10,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.437257
1,kmeans,t2.3.6+2.3.3+2.3.5,1,13,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.670915
2,kmeans,t2.3.6+2.3.3+2.3.5,1,16,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.729734
3,kmeans,t2.3.6+2.3.3+2.3.5,1,19,19,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.995792
4,kmeans,t2.3.6+2.3.3+2.3.5,1,22,22,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",2.332657
5,kmeans,t2.3.6+2.3.3+2.3.5,1,25,25,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",2.756663
6,kmeans,t2.3.6+2.3.3+2.3.5,1,28,28,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.043608
7,kmeans,t2.3.6+2.3.3+2.3.5,1,31,31,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.811321
8,kmeans,t2.3.6+2.3.3+2.3.5,1,34,34,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.926773
9,kmeans,t2.3.6+2.3.3+2.3.5,1,37,37,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",4.188998


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6+2.3.3+2.3.5,1,10,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.437257
1,kmeans,t2.3.6+2.3.3+2.3.5,1,13,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.670915
2,kmeans,t2.3.6+2.3.3+2.3.5,1,16,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.729734
3,kmeans,t2.3.6+2.3.3+2.3.5,1,19,19,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.995792
4,kmeans,t2.3.6+2.3.3+2.3.5,1,22,22,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",2.332657
5,kmeans,t2.3.6+2.3.3+2.3.5,1,25,25,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",2.756663
6,kmeans,t2.3.6+2.3.3+2.3.5,1,28,28,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.043608
7,kmeans,t2.3.6+2.3.3+2.3.5,1,31,31,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.811321
8,kmeans,t2.3.6+2.3.3+2.3.5,1,34,34,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.926773
9,kmeans,t2.3.6+2.3.3+2.3.5,1,37,37,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",4.188998


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6+2.3.3+2.3.5,1,10,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.437257
1,kmeans,t2.3.6+2.3.3+2.3.5,1,13,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.670915
2,kmeans,t2.3.6+2.3.3+2.3.5,1,16,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.729734
3,kmeans,t2.3.6+2.3.3+2.3.5,1,19,19,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.995792
4,kmeans,t2.3.6+2.3.3+2.3.5,1,22,22,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",2.332657
5,kmeans,t2.3.6+2.3.3+2.3.5,1,25,25,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",2.756663
6,kmeans,t2.3.6+2.3.3+2.3.5,1,28,28,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.043608
7,kmeans,t2.3.6+2.3.3+2.3.5,1,31,31,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.811321
8,kmeans,t2.3.6+2.3.3+2.3.5,1,34,34,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.926773
9,kmeans,t2.3.6+2.3.3+2.3.5,1,37,37,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",4.188998


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6+2.3.3+2.3.5,1,10,10,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.437257
1,kmeans,t2.3.6+2.3.3+2.3.5,1,13,13,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.670915
2,kmeans,t2.3.6+2.3.3+2.3.5,1,16,16,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.729734
3,kmeans,t2.3.6+2.3.3+2.3.5,1,19,19,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",1.995792
4,kmeans,t2.3.6+2.3.3+2.3.5,1,22,22,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",2.332657
5,kmeans,t2.3.6+2.3.3+2.3.5,1,25,25,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",2.756663
6,kmeans,t2.3.6+2.3.3+2.3.5,1,28,28,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.043608
7,kmeans,t2.3.6+2.3.3+2.3.5,1,31,31,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.811321
8,kmeans,t2.3.6+2.3.3+2.3.5,1,34,34,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",3.926773
9,kmeans,t2.3.6+2.3.3+2.3.5,1,37,37,"rt_vect: 0.22918415069580078, min_df: 4, max_d...",4.188998


In [34]:
kmeans_labels_results

,name,10,13,16,19,22,25,28,31,34,37,40,43,46,49
0,google/chaps-linux,9,2,3,14,5,5,18,25,14,16,5,16,9,4
1,droidefense/engine,2,11,10,17,15,3,24,5,12,7,28,26,26,42
2,wdgdc/Jarvis,4,0,11,11,4,21,16,23,2,33,34,21,44,0
3,PaloAltoNetworks/google-cloud-hub-spoke-tutorial,0,2,3,0,8,2,18,4,31,31,16,22,22,32
4,liwenhe1993/charts,0,2,3,14,5,2,18,25,14,30,5,28,28,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28390,sakhnyuk/javascript-interview-questions,0,2,3,5,8,2,18,4,33,32,16,22,22,45
28391,mohammaduzair9/Face-Recognition,0,9,1,14,16,5,15,11,14,16,27,10,16,12
28392,deepinx/age-gender-estimation,9,2,13,15,1,19,10,10,23,12,30,15,15,36
28393,aws-cloudformation/aws-cloudformation-resource...,2,11,12,11,15,3,24,5,12,7,28,26,26,24


In [35]:
print("saving results to disk")

try:
    old_results = from_disk(f'results/{cust_dir}/kmeans_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_labels_results_t2.3.6+2.3.3+2.3.5.pkl
